Experiments to see how to send data to XCSoar
We have added an extra port into FillTCPPorts() of 9002

We should generate fake NMEA data and send it in 
as a circle to see if it gets the radius right as a 
UDP format.  

Look at hacking EyeDevice() perhaps

1. Find default NMEA format and generate it
2. Send the NMEA in as UDP to a socket and device
3. Check if the system reports circles of the right radius
4. Check if system reports wind direction right
5. Check if device debug logs everything
6. Check if NMEA logger logs everything incl bogus strings
7. (Set thermal assist windows on flight phone)
8. Prove the orientation box can work with NMEA/Z sentences

9. Determin the bandwidth of values coming in (can this be used to log directly from the sensors
10. Find a way to plot the blue circle in a window

-- Get orientation values 

$PCPROBE,T,FD92,FF93,00D9,FD18,017E,FEDB,0370,0075,00D6,0064,001C,000000,, 

$PCPROBE,T,Q0,Q1,Q2,Q3,ax,ay,az,temp,rh,batt,delta_press,abs_press,C,

Encode into the probe value (maybe the humidity)
But start with the BNO0



In [27]:
import socket
udpsocket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
udpaddress = ("localhost", 4353)

In [111]:
import math, time, datetime

tstamp0 = datetime.datetime.now()
lat0, lng0 = 53.4121628,-2.9864259
lat0, lng0 = 53.413522,-1.9062517
velknots, veldir = 0, 0

earthrad = 6378137
nyfac = 2*math.pi*earthrad/360
exfac = nyfac*math.cos(math.radians(lat0))

def tform(dtseconds):
    tstamp = tstamp0 + datetime.timedelta(seconds=dtseconds)
    return "%02u%02u%02u.%03d" % \
        (tstamp.hour, tstamp.minute, tstamp.second, int(tstamp.microsecond/1000))
    
def tformdate(dtseconds):
    tstamp = tstamp0 + datetime.timedelta(seconds=dtseconds)
    return "%02u%02u%02u" % (tstamp.day, tstamp.month, tstamp.year%100)

    
def latlngform(x, y):
    lat = lat0 + y/nyfac
    lng = lng0 + x/exfac
    return "%02d%06.3f,%c,%02d%06.3f,%c" % \
        (int(abs(lat)), (abs(lat)-math.floor(abs(lat)))*60, 'N' if lat>0 else 'S', 
         int(abs(lng)), (abs(lng)-math.floor(abs(lng)))*60, 'E' if lng>0 else 'W')

def wrapchecksum(rec):
    s = 0
    for c in rec:
        s ^= ord(c)
    return "${:s}*{:02x}\r\n".format(rec, s).encode()

prevx, prevy, prevdtseconds = 0, 0, 0
def gprmcrec(dtseconds, x, y):
    global prevx, prevy, prevdtseconds
    dx, dy, ddt = x-prevx, y-prevy, dtseconds-prevdtseconds
    velknots = math.hypot(dx, dy)*1.94384/max(0.1, ddt)
    veldir = (360+math.degrees(math.atan2(dx, dy)))%360
    prevx, prevy, prevdtseconds = x, y, dtseconds
    #velknots, veldir = 0, 0
    rec = "GPRMC,%s,A,%s,%05.1f,%05.1f,%s,000.0," % \
        (tform(dtseconds), latlngform(x, y), 
         min(velknots, 100), veldir, tformdate(dtseconds))
    return wrapchecksum(rec)

def gpggarec(dtseconds, x, y, alt):
    rec = "GPGGA,%s,%s,1,12,1.0,%.1f,M,0.0,M,," % \
        (tform(dtseconds), latlngform(x, y), alt)
    return wrapchecksum(rec)

#$GPRMC,153412.410,A,5230.600,N,01323.246,E,2027.8,289.2,310718,000.0,W*4C
#$GPGGA,153413.410,5230.879,N,01322.443,E,1,12,1.0,0.0,M,0.0,M,,*69
gprmcrec(1, 100, 0)
gpggarec(1, 100, 0, 100)


b'$GPGGA,193345.806,5324.811,N,0154.285,W,1,12,1.0,100.0,M,0.0,M,,*4d\r\n'

In [406]:
import time, math
circlerad = 90
circleperiod = 40   
climbrate = 1.2
tperiod = 10  # readings per second

udprecs = [ ]
def ust(rec):
    udprecs.append(rec)
    udpsocket.sendto(rec, udpaddress)

for i in range(4000):
    t = i/tperiod
    theta = math.radians(360*t/circleperiod)
    x, y = math.sin(theta)*circlerad, math.cos(theta)*circlerad
    alt = 800+t*climbrate
    ust(gprmcrec(t, x, y))
    ust(gpggarec(t, x, y, alt))
    ust(wrapchecksum(CprobeNMEA(t)))
    time.sleep(1/tperiod)
    

KeyboardInterrupt: 

In [ ]:
# Make a Q-format converter
# Qt00000DFDu04D8C198y01E8D98DxFFE4C97Da01A0
# Find the default $GPGSA or whatever

In [359]:
def CProbeQuatToQEuler(q):
    def FromXY(x, y):  return math.atan2(y,x)
    def Square(x):  return x**2
    sin_pitch = -2 * (q[0] * q[2] - q[3] * q[1])
    pitch_angle = math.asin(sin_pitch)
    heading = math.pi + FromXY(Square(q[3]) - Square(q[0]) - Square(q[1]) + Square(q[2]), 2 * (q[1] * q[2] + q[3] * q[0]));
    roll = FromXY(Square(q[3]) + Square(q[0]) - Square(q[1]) - Square(q[2]), 2 * (q[0] * q[1] + q[3] * q[2]))
    return math.degrees(pitch_angle), math.degrees(heading), math.degrees(roll)

def Quat(X, Y, Z):
    yaw, roll, pitch = math.radians(Z), math.radians(X), math.radians(Y)
    cy = math.cos(yaw * 0.5);
    sy = math.sin(yaw * 0.5);
    cr = math.cos(roll * 0.5);
    sr = math.sin(roll * 0.5);
    cp = math.cos(pitch * 0.5);
    sp = math.sin(pitch * 0.5);

    w = cy * cr * cp + sy * sr * sp;
    x = cy * sr * cp - sy * cr * sp;
    y = cy * cr * sp + sy * sr * cp;
    z = sy * cr * cp - cy * sr * sp;
    return w, x, y, z
def CProbeEulerToQuat(pitch_angle, heading, roll):
    return Quat(roll, -pitch_angle, 360-heading)
CProbeQuatToQEuler(CProbeEulerToQuat(19,212,3.8))


(19.0, 212.0, 3.8000000000000007)

In [403]:
def CprobeNMEA(t):
    pitch_angle, heading, roll = 19, 212, 3.8
    acc = 0, 0, 5
    tempdegc = 28
    humidpc = 60
    battpc = 44
    delta_press = 50
    abs_press = 101000
    cprobecharging = True
    
    tempdegc = t%20
    humidpc = (t%70)+10
    roll = math.sin(t/2)*30
    heading = (t*17)%360

    q = CProbeEulerToQuat(pitch_angle, heading, roll)
    def qcot(w):  return int(w*1000)&0xFFFF
    qs = "%04X,%04X,%04X,%04X" % (qcot(q[0]), qcot(q[1]), qcot(q[2]), qcot(q[3]))
    def acot(w):  return int(w/9.81*1000)&0xFFFF
    acs = "%04X,%04X,%04X" % (acot(acc[0]), acot(acc[1]), acot(acc[2]))

    k = "PCPROBE,T,%s,%s,%04X,%04X,%04X,%04X,,%s," % \
    (qs, acs, int(tempdegc*10), int(humidpc*10), int(battpc), int(delta_press*10), "C" if cprobecharging else "")
    return k


In [401]:
wrapchecksum(CprobeNMEA(2))

b'$PCPROBE,T,00E6,00D6,00A2,03A7,0000,0000,01FD,0014,0078,002C,01F4,,C,*6e\r\n'